In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [4]:
# df0 is a mix of two datasets: "Collisions-Events" and "Collisions-Involved"
df0 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/involved2020_forinsight_r1.csv")
df0

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (12,26,29,30,31,32,34) have mixed types. Specify dtype option on import or set low_memory=False.



,collision_no,accyear,accdate,acctime,longitude,latitude,stname1,streetype1,dir1,stname2,streetype2,dir2,stname3,streetype3,dir3,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_no,vehicle_class,initial_dir,impact_type,involved_class,involved_injury_class,safety_equip_used,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,cyclist_action,cyclist_condition,cyclist_collision_type,manoeuver,posted_speed,actual_speed
0,1,2000,2000-09-11,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",NORTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING LEFT,NaN,NaN
1,1,2000,2000-09-11,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,2.0,"AUTOMOBILE, STATION WAGON",SOUTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,NaN,NaN
2,2,2000,2000-09-11,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,UNKNOWN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2000,2000-09-11,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,NaN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2000,2000-09-11,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,4.0,NaN,UNKNOWN,OTHER,VEHICLE OWNER,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING RIGHT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601707,1038396,2019,2019-05-10,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,3.0,"AUTOMOBILE, STATION WAGON",EAST,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,PARKED,40.0,0.0
2601708,1038396,2019,2019-05-10,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,2.0,NaN,NaN,TURNING MOVEMENT,PASSENGER,MINOR,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2601709,1038397,2019,2019-07-20,2236,-79.314602,43.678378,GERRARD ST E,NaN,NaN,BOWMORE RD,NaN,NaN,43 m East of,NaN,NaN,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",EAST,SMV UNATTENDED VEHICLE,DRIVER,NONE,USE UNKNOWN,OTHER,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,40.0,50.0
2601710,1038398,2019,2019-10-08,938,-79.372333,43.651519,JARVIS ST,NaN,NaN,ADELAIDE ST E,NaN,NaN,NaN,NaN,NaN,Major Arterial,CLEAR,DAYLIGHT,DRY,TRAFFIC SIGNAL,1.0,NaN,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,USE UNKNOWN,NaN,NaN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df0.columns

Index(['collision_no', 'accyear', 'accdate', 'acctime', 'longitude',
       'latitude', 'stname1', 'streetype1', 'dir1', 'stname2', 'streetype2',
       'dir2', 'stname3', 'streetype3', 'dir3', 'road_class', 'visibility',
       'light', 'road_surface_cond', 'traffic_control', 'vehicle_no',
       'vehicle_class', 'initial_dir', 'impact_type', 'involved_class',
       'involved_injury_class', 'safety_equip_used', 'driver_action',
       'driver_condition', 'pedestrian_action', 'pedestrian_condition',
       'pedestrian_collision_type', 'cyclist_action', 'cyclist_condition',
       'cyclist_collision_type', 'manoeuver', 'posted_speed', 'actual_speed'],
      dtype='object')

In [6]:
# This shows the number of collisions itself (NOT the number of people involved in the collisions)
df0['collision_no'].nunique()

1038398

In [0]:
# Create df1 from df0
df1 = df0.copy()

# Add new column 'acc_date'
df1['acc_date'] = pd.to_datetime(df0['accdate'], errors = 'raise')
df1['year'] = df1['acc_date'].dt.year
df1['month_name'] = df1['acc_date'].dt.month_name()
df1['day_name'] = df1['acc_date'].dt.day_name()

# Drop columns 'accdate' and 'accyear'
df1.drop(['accdate', 'accyear'], axis=1, inplace=True)

In [8]:
df1['acc_date'].dt.year.unique() 

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [9]:
df1['involved_injury_class'].unique()

array(['NONE', 'MINIMAL', nan, 'MINOR', 'FATAL', 'MAJOR'], dtype=object)

In [10]:
print('{:,}'.format(df1['involved_injury_class'].isna().sum()), 
      'data points in "involved_injury_class" are nan, which will be removed.')
df1.dropna(subset=['involved_injury_class'], inplace=True)

137,349 data points in "involved_injury_class" are nan, which will be removed.


In [11]:
df1['involved_class'].unique()

array(['DRIVER', 'PASSENGER', 'VEHICLE OWNER', nan, 'OTHER',
       'OTHER PROPERTY OWNER', 'TRUCK DRIVER', 'PEDESTRIAN', 'CYCLIST',
       'WITNESS', 'MOTORCYCLE DRIVER', 'RUNAWAY - NO DRIVER',
       'WHEELCHAIR', 'IN-LINE SKATER', 'MOPED DRIVER', 'UNKNOWN - FTR',
       'DRIVER - NOT HIT', 'TRAILER OWNER', 'MOTORCYCLE PASSENGER',
       'CYCLIST - NOT HIT', 'MOPED PASSENGER', 'CYCLIST PASSENGER',
       'ANIMAL', 'PEDESTRIAN - NOT HIT'], dtype=object)

In [12]:
print('{:,}'.format(df1['involved_class'].isna().sum()), 
      'data points in "involved_class" are nan, which will be removed.')
df1.dropna(subset=['involved_class'], inplace=True)

12,228 data points in "involved_class" are nan, which will be removed.


In [13]:
print('{:,}'.format(df1['longitude'].isna().sum()), 
      'data points in "longitude" are nan, which will be removed.')
df1.dropna(subset=['longitude'], inplace=True)

21 data points in "longitude" are nan, which will be removed.


In [14]:
df1['latitude'].isna().sum()

0

In [15]:
# Drop points whose lon-lat are not in Toronto (spatial outlier)
# Toronto lon-lat ranges:
#    longitude: (-79.65 , -79.08)
#    latitude: (43.57 , 43.85)

i = df1[(df1['longitude']<-79.65) | (df1['longitude']>-79.08) | (df1['latitude']<43.57) | (df1['latitude']>43.85)].index
df1.drop(i, inplace=True)

print('There were {:,} spatial outliers which are removed now.'.format(i.size))

There were 2,421 spatial outliers which are removed now.


In [16]:
# Add new columns to df1
df1['# involved'] = int(1)

df1['# injured'] = int(0)
df1.loc[df1['involved_injury_class'].isin(['MINIMAL', 'MINOR', 'MAJOR']), ['# injured']] = int(1) 

df1['# fatalities'] = int(0)
df1.loc[df1['involved_injury_class']=='FATAL', ['# fatalities']] = int(1) 

df1['# KSI'] = int(0)
df1.loc[df1['involved_injury_class'].isin(['MAJOR', 'FATAL']), ['# KSI']] = int(1) 
df1

,collision_no,acctime,longitude,latitude,stname1,streetype1,dir1,stname2,streetype2,dir2,stname3,streetype3,dir3,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_no,vehicle_class,initial_dir,impact_type,involved_class,involved_injury_class,safety_equip_used,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,cyclist_action,cyclist_condition,cyclist_collision_type,manoeuver,posted_speed,actual_speed,acc_date,year,month_name,day_name,# involved,# injured,# fatalities,# KSI
0,1,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",NORTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING LEFT,NaN,NaN,2000-09-11,2000,September,Monday,1,0,0,0
1,1,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,2.0,"AUTOMOBILE, STATION WAGON",SOUTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,NaN,NaN,2000-09-11,2000,September,Monday,1,0,0,0
2,2,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,UNKNOWN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-09-11,2000,September,Monday,1,1,0,0
3,2,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,NaN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-09-11,2000,September,Monday,1,1,0,0
4,2,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,4.0,NaN,UNKNOWN,OTHER,VEHICLE OWNER,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING RIGHT,NaN,NaN,2000-09-11,2000,September,Monday,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601707,1038396,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,3.0,"AUTOMOBILE, STATION WAGON",EAST,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,PARKED,40.0,0.0,2019-05-10,2019,May,Friday,1,0,0,0
2601708,1038396,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,2.0,NaN,NaN,TURNING MOVEMENT,PASSENGER,MINOR,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-10,2019,May,Friday,1,1,0,0
2601709,1038397,2236,-79.314602,43.678378,GERRARD ST E,NaN,NaN,BOWMORE RD,NaN,NaN,43 m East of,NaN,NaN,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",EAST,SMV UNATTENDED VEHICLE,DRIVER,NONE,USE UNKNOWN,OTHER,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,40.0,50.0,2019-07-20,2019,July,Saturday,1,0,0,0
2601710,1038398,938,-79.372333,43.651519,JARVIS ST,NaN,NaN,ADELAIDE ST E,NaN,NaN,NaN,NaN,NaN,Major Arterial,CLEAR,DAYLIGHT,DRY,TRAFFIC SIGNAL,1.0,NaN,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,USE UNKNOWN,NaN,NaN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-08,2019,October,Tuesday,1,1,0,0


In [0]:
############### In the 'Main' notebook, here dfE is created: the dataframe of "Collisions-Events" from df1 ###################################################################

In [0]:
# Create df2 from df1 and reset its index (as some rows are already dropped)
df2 = df1.copy()
df2.reset_index(inplace=True)

In [19]:
for column in df2.columns:
    print('# nan in "{}": {:,}'.format(column, df2[column].isna().sum()))

# nan in "index": 0
# nan in "collision_no": 0
# nan in "acctime": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "stname1": 6
# nan in "streetype1": 610,486
# nan in "dir1": 1,813,420
# nan in "stname2": 330,820
# nan in "streetype2": 848,876
# nan in "dir2": 2,106,067
# nan in "stname3": 2,343,532
# nan in "streetype3": 2,449,693
# nan in "dir3": 2,449,693
# nan in "road_class": 95,433
# nan in "visibility": 6,842
# nan in "light": 255
# nan in "road_surface_cond": 1,426
# nan in "traffic_control": 13,333
# nan in "vehicle_no": 48
# nan in "vehicle_class": 161,514
# nan in "initial_dir": 563,051
# nan in "impact_type": 285
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "safety_equip_used": 597,098
# nan in "driver_action": 1,291,119
# nan in "driver_condition": 1,297,659
# nan in "pedestrian_action": 2,413,049
# nan in "pedestrian_condition": 2,411,785
# nan in "pedestrian_collision_type": 2,417,365
# nan in "cyclist_action": 2,433,938
# nan in "

In [20]:
df2['manoeuver'].unique()

array(['TURNING LEFT', 'GOING AHEAD', nan, 'TURNING RIGHT',
       'SLOWING OR STOPPING', 'PULLING AWAY FROM SHOULDER OR CURB',
       'UNKNOWN', 'STOPPED', 'CHANGING LANES', 'PARKED',
       'PULLING ONTO SHOULDER OR TOWARDCURB', 'REVERSING', 'OTHER',
       'MAKING "U" TURN', 'DISABLED', 'OVERTAKING', 'MERGING',
       'STOPPED OR PARKED'], dtype=object)

In [21]:
print('In addition to {:,} nan values for "manoeuver", there are {:,} "UNKNOWN" (not "OTHER") elements in it.'
    .format(df2['manoeuver'].isna().sum(), 
            df2[df2['manoeuver']=='UNKNOWN'].shape[0]))

In addition to 260,159 nan values for "manoeuver", there are 326,385 "UNKNOWN" (not "OTHER") elements in it.


In [22]:
df2['involved_class'].unique()

array(['DRIVER', 'PASSENGER', 'VEHICLE OWNER', 'OTHER',
       'OTHER PROPERTY OWNER', 'TRUCK DRIVER', 'PEDESTRIAN', 'CYCLIST',
       'WITNESS', 'MOTORCYCLE DRIVER', 'RUNAWAY - NO DRIVER',
       'WHEELCHAIR', 'IN-LINE SKATER', 'MOPED DRIVER', 'UNKNOWN - FTR',
       'DRIVER - NOT HIT', 'TRAILER OWNER', 'MOTORCYCLE PASSENGER',
       'CYCLIST - NOT HIT', 'MOPED PASSENGER', 'CYCLIST PASSENGER',
       'ANIMAL', 'PEDESTRIAN - NOT HIT'], dtype=object)

In [23]:
df2['driver_action'].unique()

array([nan, 'DRIVING PROPERLY', 'FAILED TO YIELD RIGHT OF WAY',
       'IMPROPER LANE CHANGE', 'IMPROPER TURN',
       'DISOBEYED TRAFFIC CONTROL', 'FOLLOWING TOO CLOSE', 'OTHER',
       'LOST CONTROL', 'SPEED TOO FAST FOR CONDITION', 'IMPROPER PASSING',
       'EXCEEDING SPEED LIMIT', 'SPEED TOO SLOW',
       'WRONG WAY ON ONE WAY ROAD'], dtype=object)

In [24]:
print('There are {:,} accident points where "involved_class" is "DRIVER". But out of them {:,} "driver_action" is nan.'.
      format(df2[df2['involved_class']=='DRIVER'].shape[0], 
             df2[(df2['involved_class']=='DRIVER') & (df2['driver_action'].isna())].shape[0]))

There are 1,807,193 accident points where "involved_class" is "DRIVER". But out of them 690,989 "driver_action" is nan.


In [25]:
df2['pedestrian_action'].unique()

array([nan, 'CROSSING WITHOUT RIGHT OF WAY', 'OTHER',
       'ON SIDEWALK OR SHOULDER', 'CROSSING WITH RIGHT OF WAY',
       'COMING FROM BEHIND PARKED VEHICLE',
       'CROSSING, NO TRAFFIC CONTROL', 'RUNNING ONTO ROADWAY',
       'WALKING ON ROADWAY WITH TRAFFIC', 'PERSON GETTING ON/OFF VEHICLE',
       'CROSSING, PEDESTRIAN CROSSOVER', 'PUSHING/WORKING ON VEHICLE',
       'PLAYING OR WORKING ON HIGHWAY',
       'CROSSING MARKED CROSSWALK WITHOUT ROW',
       'WALKING ON ROADWAY AGAINST TRAFFIC',
       'PERSON GETTING ON/OFF SCHOOL BUS'], dtype=object)

In [26]:
print('There are {:,} accident points where "involved_class" is "PEDESTRIAN". But out of them {:,} "pedestrian_action" is nan.'.
      format(df2[df2['involved_class']=='PEDESTRIAN'].shape[0], 
             df2[(df2['involved_class']=='PEDESTRIAN') & (df2['pedestrian_action'].isna())].shape[0]))

There are 38,374 accident points where "involved_class" is "PEDESTRIAN". But out of them 2,165 "pedestrian_action" is nan.


In [27]:
df2['cyclist_action'].unique()

array([nan, 'FAILED TO YIELD RIGHT OF WAY', 'OTHER', 'DRIVING PROPERLY',
       'DISOBEYED TRAFFIC CONTROL', 'IMPROPER TURN', 'LOST CONTROL',
       'SPEED TOO FAST FOR CONDITION', 'IMPROPER LANE CHANGE',
       'IMPROPER PASSING', 'EXCEEDING SPEED LIMIT', 'FOLLOWING TOO CLOSE',
       'WRONG WAY ON ONE WAY ROAD', 'SPEED TOO SLOW'], dtype=object)

In [28]:
print('There are {:,} accident points where "involved_class" is "CYCLIST". But out of them {:,} "cyclist_action" is nan.'.
      format(df2[df2['involved_class']=='CYCLIST'].shape[0], 
             df2[(df2['involved_class']=='CYCLIST') & (df2['cyclist_action'].isna())].shape[0]))

There are 22,761 accident points where "involved_class" is "CYCLIST". But out of them 7,030 "cyclist_action" is nan.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [0]:
# Drop the following columns form df2 as they contain too many 'nan' and also UNKNOWN elements. 
df2.drop(['streetype1', 'dir1', 'streetype2', 'dir2', 'streetype3', 'dir3', 'initial_dir', 'safety_equip_used', 
          'driver_action', 'driver_condition', 'pedestrian_action', 'pedestrian_condition', 'pedestrian_collision_type', 'cyclist_action', 'cyclist_condition', 'cyclist_collision_type',
         'manoeuver', 'posted_speed', 'actual_speed'], axis=1, inplace=True)

In [30]:
for column in df2.columns:
    print('# nan in "{}": {:,}'.format(column, df2[column].isna().sum()))

# nan in "index": 0
# nan in "collision_no": 0
# nan in "acctime": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "stname1": 6
# nan in "stname2": 330,820
# nan in "stname3": 2,343,532
# nan in "road_class": 95,433
# nan in "visibility": 6,842
# nan in "light": 255
# nan in "road_surface_cond": 1,426
# nan in "traffic_control": 13,333
# nan in "vehicle_no": 48
# nan in "vehicle_class": 161,514
# nan in "impact_type": 285
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "acc_date": 0
# nan in "year": 0
# nan in "month_name": 0
# nan in "day_name": 0
# nan in "# involved": 0
# nan in "# injured": 0
# nan in "# fatalities": 0
# nan in "# KSI": 0


In [31]:
df2['impact_type'].unique()

array(['TURNING MOVEMENT', 'OTHER', 'SIDESWIPE', 'ANGLE', 'REAR END',
       'SMV UNATTENDED VEHICLE', 'APPROACHING', 'SMV OTHER',
       'PEDESTRIAN COLLISIONS', 'CYCLIST COLLISIONS', nan], dtype=object)

In [32]:
df2['vehicle_class'].unique()

array(['AUTOMOBILE, STATION WAGON', nan, 'TAXI', 'TRUCK - OPEN',
       'MUNICIPAL TRANSIT BUS (TTC)', 'OTHER', 'BICYCLE', 'RICKSHAW',
       'MOTORCYCLE', 'TRUCK - CLOSED (BLAZER, ETC)', 'PASSENGER VAN',
       'MOPED', 'DELIVERY VAN', 'TRUCK-TRACTOR', 'PICK UP TRUCK',
       'TRUCK (OTHER)', 'OFF ROAD - 4 WHEELS', 'TRUCK - DUMP',
       'OTHER EMERGENCY VEHICLE', 'POLICE VEHICLE', 'TOW TRUNK',
       'SCHOOL BUS', 'SCHOOL VAN', 'FIRE VEHICLE', 'OFF ROAD - 2 WHEELS',
       'STREET CAR', 'FARM TRACTOR', 'BUS (OTHER) (GO BUS, GRAY COACH)',
       'CONSTRUCTION EQUIPMENT', 'INTERCITY BUS', 'AMBULANCE',
       'TRUCK - TANK', 'OTHER SCHOOL VEHICLE/BUS', 'SNOW PLOW',
       'OFF ROAD - 3 WHEELS', 'TRUCK - CAR CARRIER',
       'MOTORIZED SNOW VEHICLE', 'MOTOR HOME', 'OTHER FARM VEHICLE',
       'OFF ROAD - OTHER', 'RAILWAY TRAIN'], dtype=object)

In [33]:
df2['traffic_control'].unique()

array(['NO CONTROL', 'TRAFFIC SIGNAL', 'STOP SIGN',
       'PEDESTRIAN CROSSOVER', 'YIELD SIGN', nan, 'TRAFFIC GATE',
       'POLICE CONTROL', 'SCHOOL GUARD', 'TRAFFIC CONTROLLER',
       'SCHOOL BUS', 'PXO - NO PED', 'STREETCAR (STOP FOR)'], dtype=object)

In [34]:
df2[df2['road_class']=='Other'].shape

(5009, 26)

In [35]:
# Create df3 from df2 for detecting most important features: Drop some unuseful columns
df3 = df2.copy()
df3.drop(['index', 'stname1', 'stname2', 'stname3', 'traffic_control', 'vehicle_no', 'vehicle_class'],axis=1, inplace=True)

# Drop 'month_name' and 'day_name' as they are string, we're going to extract the integer values from 'acc_date'
df3.drop(['year', 'month_name', 'day_name'],axis=1, inplace=True)
df3.head()

,collision_no,acctime,longitude,latitude,road_class,visibility,light,road_surface_cond,impact_type,involved_class,involved_injury_class,acc_date,# involved,# injured,# fatalities,# KSI
0,1,850,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0
1,1,850,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0
2,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0
3,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0
4,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,VEHICLE OWNER,NONE,2000-09-11,1,0,0,0


In [36]:
for column in df3.columns:
    print('# nan in "{}": {:,}'.format(column, df3[column].isna().sum()))

# nan in "collision_no": 0
# nan in "acctime": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "road_class": 95,433
# nan in "visibility": 6,842
# nan in "light": 255
# nan in "road_surface_cond": 1,426
# nan in "impact_type": 285
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "acc_date": 0
# nan in "# involved": 0
# nan in "# injured": 0
# nan in "# fatalities": 0
# nan in "# KSI": 0


In [37]:
# Drop nan rows from df3
df3.dropna(subset=['road_class', 'visibility', 'light', 'road_surface_cond', 'impact_type'], inplace=True )

for column in df3.columns:
    print('# nan in "{}": {:,}'.format(column, df3[column].isna().sum()))

# nan in "collision_no": 0
# nan in "acctime": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "road_class": 0
# nan in "visibility": 0
# nan in "light": 0
# nan in "road_surface_cond": 0
# nan in "impact_type": 0
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "acc_date": 0
# nan in "# involved": 0
# nan in "# injured": 0
# nan in "# fatalities": 0
# nan in "# KSI": 0


In [38]:
df3.reset_index(drop=True, inplace=True)
df3

,collision_no,acctime,longitude,latitude,road_class,visibility,light,road_surface_cond,impact_type,involved_class,involved_injury_class,acc_date,# involved,# injured,# fatalities,# KSI
0,1,850,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0
1,1,850,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0
2,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0
3,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0
4,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,VEHICLE OWNER,NONE,2000-09-11,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346920,1038396,2200,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,TURNING MOVEMENT,DRIVER,NONE,2019-05-10,1,0,0,0
2346921,1038396,2200,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,TURNING MOVEMENT,PASSENGER,MINOR,2019-05-10,1,1,0,0
2346922,1038397,2236,-79.314602,43.678378,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,SMV UNATTENDED VEHICLE,DRIVER,NONE,2019-07-20,1,0,0,0
2346923,1038398,938,-79.372333,43.651519,Major Arterial,CLEAR,DAYLIGHT,DRY,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,2019-10-08,1,1,0,0


In [39]:
# extract hour from acctime (integer)

hour = []
for i in range(len(df3)):
    if len(str(df3.loc[i, 'acctime'])) == 2:
        hour.append(0)
    elif len(str(df3.loc[i, 'acctime'])) == 3:
        hour.append(int(str(df3.loc[i, 'acctime'])[:1]))
    elif len(str(df3.loc[i, 'acctime'])) == 4:
        hour.append(int(str(df3.loc[i, 'acctime'])[:2]))
    else:
        hour.append(np.nan)
        
df3['hour'] = hour

df3['hour'].unique()

array([ 8.,  6., 11., 17., 13., 14., 16., 19.,  2., 18., 20., 22., 15.,
       21.,  1.,  7.,  9., 12., 23., 10.,  0., nan,  3.,  4.,  5.])

In [40]:
print('# nan in "hour": {:,}'.format(df3['hour'].isna().sum()))

# nan in "hour": 11,401


In [41]:
# Drop nan values of hour
df3.dropna(subset=['hour'], inplace=True )

df3['hour'] = df3['hour'].astype(np.int8)
df3.head()

,collision_no,acctime,longitude,latitude,road_class,visibility,light,road_surface_cond,impact_type,involved_class,involved_injury_class,acc_date,# involved,# injured,# fatalities,# KSI,hour
0,1,850,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0,8
1,1,850,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0,8
2,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0,8
3,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0,8
4,2,850,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,VEHICLE OWNER,NONE,2000-09-11,1,0,0,0,8


In [0]:
df3['year'] = df3['acc_date'].dt.year
df3['month'] = df3['acc_date'].dt.month
df3['day'] = df3['acc_date'].dt.dayofweek

In [43]:
df3['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [44]:
df3['month'].unique()

array([ 9,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12])

In [45]:
# It is assumed the week starts on Monday, which is denoted by 0 and ends on Sunday which is denoted by 6
df3['day'].unique()

array([0, 6, 1, 2, 3, 4, 5])

In [46]:
# Create df4 from df3
df4 = df3[['collision_no', 'year', 'month', 'day', 'hour', 'longitude', 'latitude', 
           'road_class', 'road_surface_cond', 'visibility', 'light', 'impact_type', 
           'involved_class', 'involved_injury_class', '# involved', '# injured', '# KSI', '# fatalities']]

df4.reset_index(drop=True, inplace=True)
df4

,collision_no,year,month,day,hour,longitude,latitude,road_class,road_surface_cond,visibility,light,impact_type,involved_class,involved_injury_class,# involved,# injured,# KSI,# fatalities
0,1,2000,9,0,8,-79.456317,43.727732,Minor Arterial,DRY,CLEAR,DAYLIGHT,TURNING MOVEMENT,DRIVER,NONE,1,0,0,0
1,1,2000,9,0,8,-79.456317,43.727732,Minor Arterial,DRY,CLEAR,DAYLIGHT,TURNING MOVEMENT,DRIVER,NONE,1,0,0,0
2,2,2000,9,0,8,-79.456271,43.727698,Minor Arterial,DRY,CLEAR,DAYLIGHT,OTHER,PASSENGER,MINIMAL,1,1,0,0
3,2,2000,9,0,8,-79.456271,43.727698,Minor Arterial,DRY,CLEAR,DAYLIGHT,OTHER,PASSENGER,MINIMAL,1,1,0,0
4,2,2000,9,0,8,-79.456271,43.727698,Minor Arterial,DRY,CLEAR,DAYLIGHT,OTHER,VEHICLE OWNER,NONE,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335519,1038396,2019,5,4,22,-79.351940,43.677554,Major Arterial,DRY,CLEAR,"DARK, ARTIFICIAL",TURNING MOVEMENT,DRIVER,NONE,1,0,0,0
2335520,1038396,2019,5,4,22,-79.351940,43.677554,Major Arterial,DRY,CLEAR,"DARK, ARTIFICIAL",TURNING MOVEMENT,PASSENGER,MINOR,1,1,0,0
2335521,1038397,2019,7,5,22,-79.314602,43.678378,Minor Arterial,WET,RAIN,"DARK, ARTIFICIAL",SMV UNATTENDED VEHICLE,DRIVER,NONE,1,0,0,0
2335522,1038398,2019,10,1,9,-79.372333,43.651519,Major Arterial,DRY,CLEAR,DAYLIGHT,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,1,1,0,0


In [47]:
df4.isna().sum()

collision_no             0
year                     0
month                    0
day                      0
hour                     0
longitude                0
latitude                 0
road_class               0
road_surface_cond        0
visibility               0
light                    0
impact_type              0
involved_class           0
involved_injury_class    0
# involved               0
# injured                0
# KSI                    0
# fatalities             0
dtype: int64

In [48]:
df4[df4['road_class']=='Pending'].shape

(1030, 18)

In [49]:
df4[df4['road_class']==' '].shape

(5, 18)

In [50]:
i = df4[df4['road_class'] == 'Pending'].index
df4.drop(i, inplace=True)

df4.reset_index(drop=True, inplace=True)

i = df4[df4['road_class'] == ' '].index
df4.drop(i, inplace=True)

df4.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
df_inKsi = df4[['involved_class', '# KSI']].groupby(['involved_class']).sum()
df_inKsi.reset_index(inplace=True)
df_inKsi.sort_values('# KSI', inplace=True, ascending=False)
df_inKsi

,involved_class,# KSI
14,PEDESTRIAN,3999
4,DRIVER,2681
13,PASSENGER,1359
1,CYCLIST,938
9,MOTORCYCLE DRIVER,644
18,TRUCK DRIVER,31
7,MOPED DRIVER,28
10,MOTORCYCLE PASSENGER,23
21,WHEELCHAIR,17
11,OTHER,7


In [55]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Bar(
    x = df_inKsi['involved_class'], 
    y = df_inKsi['# KSI'],
    # text = feature_imp,
    # textposition = 'outside',
    # texttemplate = '%{text:.2f}'+'%'
    )])
fig.update_layout(title = '',
                  xaxis = dict(
                      title = 'Involved People'), 
                  yaxis = dict(
                      title = '# KSI'
                  ))
fig.show()

In [0]:
# ohe stands for one-hot encoding
ohe_fields=['road_class', 'road_surface_cond', 'visibility', 'light', 'impact_type', 'involved_class', 'involved_injury_class']

# One-Hot encode a couple of variables
df4 = pd.get_dummies(df4, columns=ohe_fields, prefix=ohe_fields)
df4

In [0]:
# df4.to_csv('df4.csv')